# CourtRankRL GRPO Training - Chunk-Based, A100 Memory Efficient

## Agents.md Specifikáció (Chunk-Based)

Ez a notebook a CourtRankRL GRPO alapú reranking modell tanítását végzi el **A100 GPU-n** (80GB VRAM, memory-constrained).

### Főbb jellemzők (Chunk-Based megoldás):
- **Model**: Qwen/Qwen3-4B-Instruct-2507 (4-bit) + QLoRA (rank=32, alpha=64)
- **Training**: TRL GRPOTrainer GRPO algoritmussal
  - Loss: "dapo" (eliminates length bias)
  - Reward scaling: "batch" (robust - PPO Lite)
  - Importance sampling: "sequence" (stable - GSPO)
- **Dataset**: 98 query (teljes), 20 chunk/slate, **TELJES chunk szöveg** (~500-800 char)
- **Slate stratégia**: Chunk-level retrieval (nem doc aggregáció!) → legrelevánsabb chunk-ok
- **Baseline**: Slate sorrendje = fusion ranking [0,1,2,...] (BM25+FAISS fusion szerint)
- **Hardware**: Batch size 2, grad accumulation 6, 6 generations/prompt (MEMORY EFFICIENT)
- **Training time**: ~25-35 perc (600 steps, vLLM-mel, csökkentett gen miatt)

### Miért chunk-based?
- ✅ **Releváns kontextus**: BM25+FAISS már kiválasztotta a legrelevánsabb chunk-okat
- ✅ **Teljes szöveg**: A model látja, MIÉRT releváns egy dokumentum
- ✅ **Jobb tanulás**: A model megtanulja értékelni a valódi tartalmat, nem csak metaadatokat


In [ ]:
# Környezet setup és csomagok telepítése
# KRITIKUS: Kompatibilis verziók telepítése (Unsloth + TRL + dependencies)

# 1. Core dependencies ELŐSZÖR (stabil verziók)
%pip install -q --upgrade pip
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers datasets huggingface_hub
%pip install -q numpy scipy scikit-learn

# 2. Accelerate - FIXÁLT VERZIÓ az is_fsdp2 kompatibilitás miatt
# CRITICAL: accelerate>=0.30.0 szükséges az is_fsdp2 attribútumhoz, de kompatibilisnek kell lennie Unsloth-tal
%pip install -q "accelerate>=0.30.0" --upgrade

# 3. LoRA és kvantizáció
%pip install -q peft bitsandbytes

# 4. TRL (legfrissebb stabil verzió - GRPO támogatással)
%pip install -q --upgrade trl

# 5. Unsloth telepítése UTOLJÁRA (hogy a TRL-hez igazodjon)
%pip install -q --upgrade unsloth

# 6. vLLM (opcionális, de ajánlott fast_inference-hez)
%pip install -q vllm

# 7. Egyéb függőségek
%pip install -q --upgrade typing_extensions ranx

print("✅ Csomagok telepítve (kompatibilis verziók)")
print("📦 Telepített verziók ellenőrzése:")
import unsloth, torch, transformers, trl, accelerate
print(f"  - PyTorch: {torch.__version__}")
print(f"  - Transformers: {transformers.__version__}")
print(f"  - Accelerate: {accelerate.__version__} (CRITICAL: >=0.30.0 az is_fsdp2 támogatáshoz)")
print(f"  - TRL: {trl.__version__}")
print(f"  - Unsloth: {unsloth.__version__}")

# FONTOS FIGYELMEZTETÉS a verzióról
if tuple(map(int, accelerate.__version__.split('.')[:2])) < (0, 30):
    print(f"⚠️  FIGYELEM: Accelerate verzió ({accelerate.__version__}) < 0.30.0")
    print(f"   A monkey patch fog futni a training során az is_fsdp2 kompatibilitásért")
    print(f"   Ajánlott: pip install accelerate>=0.30.0 --upgrade && RESTART KERNEL")


In [ ]:
# Importok
import os, warnings
os.environ["TORCHDYNAMO_DISABLE"] = "1"      # hivatalos flag a dynamo kikapcsolására
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # ártalmatlan, de segít fallbackben
# (opcionális, ha Unsloth figyeli)
os.environ["UNSLOTH_COMPILE"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", message=".*An output with one or more elements was resized.*")

import json
import sys
import re
import random
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl.trainer.grpo_trainer import GRPOTrainer
from trl.trainer.grpo_config import GRPOConfig
from huggingface_hub import login
from sklearn.metrics import ndcg_score
from scipy.stats import entropy as scipy_entropy
from sklearn.model_selection import train_test_split
from ranx import Qrels, Run, evaluate

print("✅ Importok betöltve (Unsloth + TRL + sklearn + scipy + ranx)")
print(f"PyTorch verzió: {torch.__version__}")
print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# CRITICAL FIX: Accelerate is_fsdp2 kompatibilitási patch
# Ha a jelenlegi Accelerate verzió nem támogatja az is_fsdp2 attribútumot, patcheljük
try:
    from accelerate import Accelerator
    from accelerate.state import AcceleratorState
    
    # Ellenőrizzük, hogy létezik-e az is_fsdp2 property
    test_state = AcceleratorState()
    _ = test_state.is_fsdp2
    print("✅ Accelerate verzió támogatja az is_fsdp2-t")
except AttributeError:
    print("⚠️ Accelerate verzió NEM támogatja az is_fsdp2-t, monkey patch alkalmazása...")
    
    # Monkey patch: adjuk hozzá az is_fsdp2 property-t az AcceleratorState-hez
    def is_fsdp2_property(self):
        """Visszakompatibilitási patch: is_fsdp2 mindig False régebbi Accelerate verziókban"""
        return False
    
    # Property hozzáadása az osztályhoz
    AcceleratorState.is_fsdp2 = property(is_fsdp2_property)
    
    # Ellenőrzés
    test_state = AcceleratorState()
    assert hasattr(test_state, 'is_fsdp2'), "Patch sikertelen!"
    assert test_state.is_fsdp2 == False, "Patch helytelen értéket ad!"
    print("✅ Monkey patch sikeresen alkalmazva - is_fsdp2 = False")
except Exception as e:
    print(f"❌ Váratlan hiba az is_fsdp2 ellenőrzése során: {e}")
    print("⚠️ Folytatás patch nélkül - ha training hiba lesz, telepítsd újra: pip install accelerate>=0.30.0")

In [ ]:
# HuggingFace bejelentkezés
hf_token = os.getenv("HUGGINGFACE_TOKEN")
if hf_token:
    login(token=hf_token)
    print("✅ HuggingFace bejelentkezés sikeres")
else:
    print("⚠️ Nincs HUGGINGFACE_TOKEN, a modell letöltése korlátozott lehet")


In [ ]:
# Konfiguráció - MEMORY EFFICIENT (A100 80GB) + CURRICULUM LEARNING
MODEL_NAME = "unsloth/Qwen3-4B-Instruct-2507"
SLATE_SIZE = 30  # Enhanced: 20 → 30 (more documents per slate, better quality)

# Curriculum Learning konfiguráció
CURRICULUM_ENABLED = True  # Engedélyezve - LEGNAGYOBB JAVULÁSI POTENCIÁL
CURRICULUM_STRATEGY = "weighted_sampling"  # "filtered_dataset" vagy "weighted_sampling"
# weighted_sampling: könnyebb példákat gyakrabban választjuk ki
# Ez biztosítja, hogy a modell fokozatosan megtanulja jól rangsorolni
GROUP_SIZE = 8
LORA_RANK = 32  # 64→32: csökkentett rank a memória miatt
LORA_ALPHA = 64  # 128→64: alpha követi a rank-ot
LORA_DROPOUT = 0.05
MAX_SEQ_LENGTH = 16384  # 8192→12288: növelt érték a hosszabb promptokhoz (9600 token + margó)
GPU_MEMORY_UTILIZATION = 0.5  # 0.88→0.5: DRASTIKUSAN csökkentve a rendelkezésre álló szabad memória miatt
USE_GRADIENT_CHECKPOINTING = "unsloth"

LEARNING_RATE = 1e-4  # Enhanced: 5e-5 → 1e-4 (larger steps, better convergence)
MAX_STEPS = 1000  # Enhanced: 600 → 1000 (more epochs, but with early stopping if needed)
SAVE_STEPS = 200  # Save more frequently for checkpoint recovery
EVAL_STEPS = 50
LOGGING_STEPS = 10
WARMUP_STEPS = 100  # Enhanced: 50 → 100 (gentler warmup)
GRADIENT_ACCUMULATION_STEPS = 6  # 3→6: nagyobb accumulation, kisebb batch
NUM_GENERATIONS = 6  # 14→6: csökkentett generációk
PER_DEVICE_BATCH_SIZE = 2  # 4→2: FELÉRE csökkentve
# KRITIKUS: generation_batch_size-nak oszthatónak KELL lennie:
#   1. NUM_GENERATIONS-szel (6)
#   2. PER_DEVICE_BATCH_SIZE-zal (2)
# Legkisebb közös többszörös (LCM): 6
GENERATION_BATCH_SIZE = 6  # 6÷6=1, 6÷2=3 ✓
OPTIMIZER_NAME = "paged_adamw_8bit"
LR_SCHEDULER_TYPE = "linear"  # Enhanced: cosine → linear (more stable final convergence)

NDCG_K = 10
ENTROPY_BONUS = 0.01
# Reward clipping range (updated in reward function)
REWARD_CLIP_MIN = -1.0  # Will be updated in enhanced reward function
REWARD_CLIP_MAX = 2.0   # Enhanced: wider range for positive improvements
TRAIN_SPLIT = 0.8
SEED = 42

BASE_PATH = Path(os.getenv("WORKSPACE_PATH", "/workspace"))

# HuggingFace token definiálása (HF_TOKEN = hf_token)
hf_token = os.getenv("HUGGINGFACE_TOKEN")
HF_TOKEN = hf_token

BASELINE_METRICS_PATH = BASE_PATH / "baseline_query_metrics.csv"
SLATE_FILE = BASE_PATH / "training_slates.jsonl"
OUTPUT_DIR = BASE_PATH / "artifacts" / "grpo_policy"
METRICS_FILE = OUTPUT_DIR / "metrics.json"

print("📋 A100 80GB - Memory Efficient Konfiguráció:")
print(f"  Model: {MODEL_NAME}")
print(f"  LoRA Rank: {LORA_RANK} (csökkentve memória miatt)")
print(f"  GPU Memory Utilization: {GPU_MEMORY_UTILIZATION} (0.5 = 50%)")
print(f"  Batch: {PER_DEVICE_BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} (effective)")
print(f"  Generations: {NUM_GENERATIONS}, Generation batch: {GENERATION_BATCH_SIZE}")
print(f"  Steps per generation: {GENERATION_BATCH_SIZE // PER_DEVICE_BATCH_SIZE}")
print(f"  Steps: {MAX_STEPS}")
print(f"  ⚠️  Memória optimalizált beállítások aktívak!")

if not SLATE_FILE.exists():
    raise FileNotFoundError(f"❌ Slate fájl nem található: {SLATE_FILE}")

if not BASELINE_METRICS_PATH.exists():
    raise FileNotFoundError(f"❌ Baseline metrikák fájl nem található: {BASELINE_METRICS_PATH}")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# GPU memória tisztítása
import gc
import torch

# CUDA cache tisztítása
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✅ CUDA cache törölve")
    
    # Memória státusz
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    free = (total - allocated)
    
    print(f"\n📊 GPU Memória Státusz (A100 80GB):")
    print(f"  Total: {total:.2f} GB")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")
    print(f"  Free: {free:.2f} GB")
    print(f"  Free %: {(free/total)*100:.1f}%")

In [ ]:
# Segédfüggvények
def calculate_ndcg(ranked_indices: List[int], true_relevance: List[float], k: int = 10) -> float:
    if not true_relevance or not ranked_indices or max(true_relevance) == 0:
        return 0.0
    y_true = np.array(true_relevance)
    max_score = len(ranked_indices)
    y_score = np.zeros_like(y_true, dtype=float)
    for i, idx in enumerate(ranked_indices[:k]):
        if idx < len(y_true):
            y_score[idx] = max_score - i
    if np.sum(y_score) == 0:
        return 0.0
    try:
        return float(ndcg_score(y_true.reshape(1, -1), y_score.reshape(1, -1), k=k))
    except:
        return 0.0

def parse_model_ranking(completion: str, slate_size: int = SLATE_SIZE) -> List[int]:
    """
    Megengedőbb parsing - kevésbé szigorú formátum követelmény.
    Ha nincs elég valid index, kiegészítjük a hiányzókat a baseline order szerint.
    """
    try:
        # Próbálunk számokat kinyerni különböző módon
        numbers = []
        
        # 1. Vesszővel elválasztott számok
        parts = completion.split(",")
        for part in parts:
            part = part.strip()
            # Kérjük el minden számot (akár több szám egy cellában)
            for word in part.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size:
                        numbers.append(num)
        
        # 2. Ha nincs elég, próbálunk szóközzel elválasztott formátumot
        if len(numbers) < slate_size // 3:
            for word in completion.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size and num not in numbers:
                        numbers.append(num)
        
        # 3. Valid számok gyűjtése
        valid_numbers = []
        seen = set()
        for n in numbers:
            if n not in seen:
                valid_numbers.append(n)
                seen.add(n)
        
        # 4. Ha van elég valid szám (legalább 1/3-a a slate-nek), használjuk
        if len(valid_numbers) >= max(1, slate_size // 3):
            # Kiegészítjük a hiányzó indexeket a baseline order szerint
            missing_indices = [i for i in range(slate_size) if i not in valid_numbers]
            # A hiányzókat a végére teszük (nem zavarjuk meg a modell rankingjét)
            result = valid_numbers + missing_indices
            return result[:slate_size]
        
    except Exception as e:
        pass
    
    # Fallback: baseline order (nem random shuffle - így jobb mint teljesen random)
    return list(range(slate_size))

def create_training_prompt(query_id: str, slate: List[Dict]) -> str:
    """
    Enhanced prompt generation for GRPO training with explicit relevance labels and few-shot examples.
    The prompt uses explicit instructions in ENGLISH to help the model learn proper relevance-based ranking.
    Only the data content (queries, document texts, metadata) remains in HUNGARIAN as it comes from the dataset.
    
    Improvements:
    1. Explicit relevance labels (0/1/2) for each document
    2. Few-shot example ranking output
    3. More detailed relevance criteria
    4. Clearer output format specification
    """
    prompt = f'''# Legal Document Relevance Assessment and Ranking

## TASK
You are helping to rank Hungarian court decision documents by their relevance to a search query.

## SEARCH QUERY
"{query_id}"

## WHAT TO CONSIDER

When ranking documents, think about:
- How well the document topic matches the query
- Whether the content directly addresses what the query is asking
- The legal domain and court type (may or may not be relevant)
- The relevance labels (0=not relevant, 1=relevant, 2=highly relevant) - these can guide you
- BM25 and FAISS scores show some similarity signals, but content is most important

## DOCUMENT EVALUATION

You need to evaluate the following {len(slate)} document excerpts:

'''
    
    separator_line = "\u2500" * 63
    
    # Enhanced document presentation with EXPLICIT RELEVANCE LABELS
    for idx, doc in enumerate(slate):
        chunk_text = doc.get('text', '')[:800]
        bm25 = doc.get('bm25_score', 0)
        faiss = doc.get('faiss_score', 0)
        relevance = doc.get('relevance', 0)
        court = doc.get('court', 'N/A')
        domain = doc.get('domain', 'N/A')
        year = doc.get('year', 'N/A')
        
        # Relevance label mapping
        rel_label = "NOT RELEVANT" if relevance == 0 else ("RELEVANT" if relevance == 1 else "HIGHLY RELEVANT")
        
        prompt += f'''{separator_line}
DOCUMENT [{idx}]

Relevance: {rel_label} (Grade {relevance}/2)
ID: {doc.get('doc_id', 'N/A')} | Court: {court} | Domain: {domain} | Year: {year}
Scores: BM25={bm25:.2f}, FAISS={faiss:.3f}

Content:
{chunk_text}

'''
    
    # Strict ranking instructions to force tangible outputs
    prompt += f'''{separator_line}

## RANKING INSTRUCTIONS

You need to rank the {len(slate)} documents by their relevance to the search query.

### GUIDELINES (focus on relevance):
- Consider the search query and how well each document matches it
- Documents with higher RELEVANCE grades (2 > 1 > 0) are generally more relevant, but also analyze the content
- Think about topic match, content quality, and legal domain relevance
- BM25 and FAISS scores can help, but focus primarily on content relevance

### OUTPUT FORMAT (STRICT):
- Respond with EXACTLY one line in this format → RANKING: <comma-separated indices> #END
- Use document indices 0 to {len(slate)-1}, each exactly once, separated by commas and without extra spaces
- Do NOT include headings, bullet points, Markdown, or explanations before or after the line
- If you are uncertain, still provide your best full ranking covering every index
- End the line with "#END" to confirm completion

Example:
RANKING: 3,0,5,1,4,2,8,6,7,9 #END

RANKING:
'''
    
    return prompt

print("✅ Segédfüggvények definiálva")
print("  📝 Enhanced learning-to-rank prompt template:")
print("     - English instructions with Hungarian data")
print("     - Explicit ranking criteria")
print("     - Step-by-step guidance")
print("     - Clear do's and don'ts")
print("     - Structured document presentation")



In [ ]:
# Slate adatok betöltése
print(f"📂 Slate adatok betöltése: {SLATE_FILE}")
df_slates = pd.read_json(SLATE_FILE, lines=True, encoding='utf-8')
slates_data = df_slates.to_dict('records')
print(f"✅ Betöltve: {len(slates_data)} slate")

sample = slates_data[0]
print(f"\n📋 Minta slate struktúra:")
print(f"  Query ID: {sample['query_id'][:50]}...")
print(f"  Slate elemek: {len(sample['slate'])}")


In [ ]:
# Test prompt megtekintése
test_prompt = create_training_prompt(slates_data[0]["query_id"], slates_data[0]["slate"])
print("📝 RÉSZLETES GRPO PROMPT - TELJES PÉLDA")
print("="*80)
print(test_prompt)
print("="*80)
print(f"\n📊 Prompt statisztikák:")
print(f"  Teljes hossz: {len(test_prompt)} karakter")
print(f"  Becsült tokenek: ~{len(test_prompt)//4} token")
print(f"  Dokumentumok száma: {len(slates_data[0]['slate'])}")
print(f"  Slate ID: {slates_data[0]['query_id'][:60]}...")

## 🎯 Multi-Metrika Reward Setup

A standard nDCG@10 optimalizáció helyett **konvex kombinációt** alkalmazunk:
- **nDCG@10** (50%): Rangsor minőség
- **MRR@5** (35%): Első releváns találat pozíciója (üzleti prioritás!)
- **Recall@20** (15%): Completeness

Ez biztosítja, hogy a model a **valódi üzleti értéket** maximalizálja.

In [ ]:
# Multi-Metrika Reward Helper Functions (inline definiálva - nincs külső import)

from collections import Counter
from typing import Optional, Tuple

def compute_mrr(relevances: List[int], k: Optional[int] = None) -> float:
    """
    Mean Reciprocal Rank számítása.
    
    MRR@k = 1 / rank_first_relevant (ha van releváns a top-k-ban)
           = 0 (ha nincs releváns a top-k-ban)
    """
    if k is not None:
        relevances = relevances[:k]
    
    for i, rel in enumerate(relevances, start=1):
        if rel > 0:
            return 1.0 / i
    
    return 0.0


def compute_recall(relevances: List[int], k: int, total_relevant: Optional[int] = None) -> float:
    """
    Recall@k számítása.
    
    Recall@k = |{relevant docs in top-k}| / |{all relevant docs}|
    """
    if not relevances:
        return 0.0
    
    # Relevantes doc-ok száma top-k-ban
    relevant_in_topk = sum(1 for rel in relevances[:k] if rel > 0)
    
    # Összes releváns (ha nincs megadva, akkor az egész listából számoljuk)
    if total_relevant is None:
        total_relevant = sum(1 for rel in relevances if rel > 0)
    
    if total_relevant == 0:
        return 0.0
    
    return relevant_in_topk / total_relevant


def compute_ndcg_metric(relevances: List[int], k: int) -> float:
    """
    nDCG@k számítása sklearn használatával.
    
    Args:
        relevances: Relevancia értékek ranking sorrendben [rel_1, rel_2, ...]
        k: Top-k truncation
    
    Returns:
        nDCG@k érték [0, 1]
    """
    if not relevances or k <= 0:
        return 0.0
    
    # sklearn ndcg_score: [true_relevances], [predicted_scores]
    # predicted_scores: magasabb score = jobb ranking (fordított sorrend)
    relevances_truncated = relevances[:k]
    scores = list(range(len(relevances_truncated), 0, -1))  # [n, n-1, ..., 1]
    
    try:
        ndcg = ndcg_score(
            y_true=[relevances_truncated],
            y_score=[scores],
            k=k
        )
        return float(ndcg)
    except Exception:
        # Edge case: nincs releváns dokumentum
        return 0.0


def compute_multi_metric_reward(
    predicted_indices: List[int],
    slate: List[Dict],
    baseline_metrics: Dict[str, float],
    weights: Optional[Dict[str, float]] = None,
    clip_range: Tuple[float, float] = (-1.0, 1.0),
    use_sigmoid: bool = True,
    mrr_tiebreak_bonus: float = 0.02,
    diversity_penalty_weight: float = 0.0
) -> Tuple[float, Dict[str, float]]:
    """
    Multi-metrika reward számítása sigmoid-alapú stabilizációval.
    
    Javítások:
    - Sigmoid transzformáció a delta-kra → stabilabb gradiens
    - MRR tie-break bonus: ha nDCG/Recall stagnál de MRR javul
    - Diverzitás büntetés: ugyanazon court/domain túlsúly a top-10-ben
    
    Args:
        predicted_indices: Model által prediktált ranking indexek
        slate: Slate adatok [{"relevance": int, "court": str, "domain": str, ...}, ...]
        baseline_metrics: Baseline metrikák {"ndcg@10": float, "mrr@5": float, ...}
        weights: Metrika súlyok {"ndcg10": 0.6, "mrr5": 0.3, "recall20": 0.1}
        clip_range: Reward clipping tartomány (min, max)
        use_sigmoid: Ha True, sigmoid(delta) transzformáció
        mrr_tiebreak_bonus: Bonus ha csak MRR javul (+ε)
        diversity_penalty_weight: Diverzitás büntetés súlya (0 = kikapcsolva)
    
    Returns:
        (total_reward, components_dict)
    """
    # Default weights (updated: 0.6/0.3/0.1)
    if weights is None:
        weights = {"ndcg10": 0.60, "mrr5": 0.30, "recall20": 0.10}
    
    # Relevancia array a predicted ranking szerint
    relevances = [slate[i]["relevance"] for i in predicted_indices]
    
    # Policy metrikák
    policy_ndcg10 = compute_ndcg_metric(relevances, k=10)
    policy_mrr5 = compute_mrr(relevances, k=5)
    
    # Recall@20: total_relevant a teljes slate-ből (fixált candidate pool!)
    total_relevant = sum(1 for doc in slate if doc.get("relevance", 0) > 0)
    policy_recall20 = compute_recall(relevances, k=20, total_relevant=total_relevant)
    
    # Baseline metrikák
    baseline_ndcg10 = baseline_metrics.get("ndcg@10", 0.0)
    baseline_mrr5 = baseline_metrics.get("mrr@5", 0.0)
    baseline_recall20 = baseline_metrics.get("recall@20", 0.0)
    
    # Delta számítás
    delta_ndcg10 = policy_ndcg10 - baseline_ndcg10
    delta_mrr5 = policy_mrr5 - baseline_mrr5
    delta_recall20 = policy_recall20 - baseline_recall20
    
    # Sigmoid transzformáció (stabilabb reward signal)
    if use_sigmoid:
        # sigmoid(x) = 1 / (1 + exp(-k*x)), k=5 → érzékenység
        def sigmoid_transform(delta, k=5.0):
            return 2.0 / (1.0 + np.exp(-k * delta)) - 1.0  # range: [-1, 1]
        
        component_ndcg = weights["ndcg10"] * sigmoid_transform(delta_ndcg10)
        component_mrr = weights["mrr5"] * sigmoid_transform(delta_mrr5)
        component_recall = weights["recall20"] * sigmoid_transform(delta_recall20)
    else:
        # Lineáris (original)
        component_ndcg = weights["ndcg10"] * delta_ndcg10
        component_mrr = weights["mrr5"] * delta_mrr5
        component_recall = weights["recall20"] * delta_recall20
    
    # Total reward
    reward_raw = component_ndcg + component_mrr + component_recall
    
    # ====== TIE-BREAK BONUS: MRR javulás ======
    # Ha nDCG/Recall nem változik jelentősen, de MRR javul → felhasználói élmény++
    if abs(delta_ndcg10) < 0.01 and abs(delta_recall20) < 0.01 and delta_mrr5 > 0.05:
        reward_raw += mrr_tiebreak_bonus
    
    # ====== DIVERZITÁS BÜNTETÉS ======
    # Ha a top-10 túl homogén (ugyanaz a court/domain dominál) → -ε
    if diversity_penalty_weight > 0:
        top10_indices = predicted_indices[:10]
        top10_courts = [slate[i].get("court", "N/A") for i in top10_indices if i < len(slate)]
        top10_domains = [slate[i].get("domain", "N/A") for i in top10_indices if i < len(slate)]
        
        # Shannon entropy (magas = diverzitás, alacsony = homogén)
        def shannon_entropy(items):
            counts = Counter(items)
            total = len(items)
            if total == 0:
                return 0.0
            probs = [c/total for c in counts.values()]
            return -sum(p * np.log(p + 1e-9) for p in probs)
        
        court_entropy = shannon_entropy(top10_courts)
        domain_entropy = shannon_entropy(top10_domains)
        
        # Normalizálás: max entropy = log(10) ≈ 2.3
        max_entropy = np.log(10)
        diversity_score = (court_entropy + domain_entropy) / (2 * max_entropy)
        
        # Büntetés ha alacsony diverzitás (< 0.5 → homogén)
        if diversity_score < 0.5:
            penalty = diversity_penalty_weight * (0.5 - diversity_score)
            reward_raw -= penalty
    
    # Clipping
    reward_clipped = np.clip(reward_raw, clip_range[0], clip_range[1])
    
    # Komponensek dictionary (részletes tracking)
    components = {
        "reward_total": float(reward_clipped),
        "reward_raw": float(reward_raw),
        "component_ndcg10": float(component_ndcg),
        "component_mrr5": float(component_mrr),
        "component_recall20": float(component_recall),
        "delta_ndcg10": float(delta_ndcg10),
        "delta_mrr5": float(delta_mrr5),
        "delta_recall20": float(delta_recall20),
        "policy_ndcg10": float(policy_ndcg10),
        "policy_mrr5": float(policy_mrr5),
        "policy_recall20": float(policy_recall20),
        "baseline_ndcg10": float(baseline_ndcg10),
        "baseline_mrr5": float(baseline_mrr5),
        "baseline_recall20": float(baseline_recall20),
    }
    
    return float(reward_clipped), components

print("✅ Multi-Metrika Reward Helper Functions definiálva (inline)")
print(f"   🎯 Multi-metric reward: sigmoid-based, 0.6/0.3/0.1 weights")
print(f"   📦 Nincs külső import - minden inline definiálva")

In [ ]:
# Baseline metrikák betöltése (pre-computed from baseline_evaluation.ipynb)
BASELINE_METRICS_PATH = BASE_PATH / "baseline_query_metrics.csv"

if not BASELINE_METRICS_PATH.exists():
    print(f"⚠️  FIGYELEM: Baseline metrics fájl nem található: {BASELINE_METRICS_PATH}")
    print(f"   Előbb futtasd le a baseline_evaluation.ipynb notebookot!")
    print(f"   Fallback: Baseline metrikák inicializálása 0-ra (NEM AJÁNLOTT)")
    baseline_metrics_dict = {}
else:
    baseline_df = pd.read_csv(BASELINE_METRICS_PATH, encoding='utf-8')
    baseline_metrics_dict = {
        row["query_id"]: row.to_dict()
        for _, row in baseline_df.iterrows()
    }
    print(f"✅ Baseline metrikák betöltve: {len(baseline_metrics_dict)} query")
    
    # Statisztikák
    if baseline_metrics_dict:
        sample_query = list(baseline_metrics_dict.keys())[0]
        sample_metrics = baseline_metrics_dict[sample_query]
        print(f"\n📊 Elérhető metrikák (példa query):")
        for key in sorted(sample_metrics.keys()):
            if key != "query_id" and isinstance(sample_metrics[key], (int, float)):
                print(f"  • {key}: {sample_metrics[key]:.4f}")
        
        # Aggregált baseline teljesítmény
        avg_ndcg10 = np.mean([m.get("ndcg@10", 0.0) for m in baseline_metrics_dict.values()])
        avg_mrr5 = np.mean([m.get("mrr@5", 0.0) for m in baseline_metrics_dict.values()])
        avg_recall20 = np.mean([m.get("recall@20", 0.0) for m in baseline_metrics_dict.values()])
        
        print(f"\n🎯 Baseline teljesítmény (átlag):")
        print(f"  • nDCG@10:   {avg_ndcg10:.4f}")
        print(f"  • MRR@5:     {avg_mrr5:.4f}")
        print(f"  • Recall@20: {avg_recall20:.4f}")

In [ ]:
# ====== CURRICULUM LEARNING - Legnagyobb javítási potenciál ======
# Stratégia: Először könnyű példákkal tanítunk (magas baseline nDCG), aztán fokozatosan nehezebbekre
# Ez segít a modellnek először megtanulni a jó rangsorolást, aztán finomhangolni

print(f"\n📚 Dataset előkészítése CURRICULUM LEARNING-nel...")
print("="*80)

# 1. Baseline nDCG kiszámítása minden slate-hez
print("🎯 1. Baseline nDCG kiszámítása slate-enként...")
slate_baseline_ndcgs = []

for slate_data in slates_data:
    slate = slate_data.get("slate", [])
    if not slate:
        continue
    
    relevances = [doc.get('relevance', 0) for doc in slate]
    baseline_indices = list(range(len(slate)))
    baseline_ndcg = calculate_ndcg(baseline_indices, relevances, k=NDCG_K)
    
    slate_baseline_ndcgs.append({
        "query_id": slate_data["query_id"],
        "baseline_ndcg": baseline_ndcg,
        "num_relevant": sum(1 for r in relevances if r >= 1),
        "num_high_relevant": sum(1 for r in relevances if r == 2),
    })

# 2. Curriculum szintek definiálása baseline nDCG alapján
ndcgs = [s["baseline_ndcg"] for s in slate_baseline_ndcgs]

print(f"\n📊 Baseline nDCG statisztikák:")
print(f"  Átlag: {np.mean(ndcgs):.4f}")
print(f"  Median: {np.median(ndcgs):.4f}")
print(f"  Min: {np.min(ndcgs):.4f}, Max: {np.max(ndcgs):.4f}")
print(f"  Szórás: {np.std(ndcgs):.4f}")

# KRITIKUS: Ellenőrizzük hogy van-e elég szórás a curriculum besoroláshoz
ndcg_range = np.max(ndcgs) - np.min(ndcgs)
VARIANCE_THRESHOLD = 0.05  # Ha a tartomány kisebb mint ez, fallback stratégia

if ndcg_range < VARIANCE_THRESHOLD:
    print(f"\n⚠️  FIGYELEM: nDCG eloszlás összeomlott! (range={ndcg_range:.4f} < {VARIANCE_THRESHOLD})")
    print(f"   Valószínű ok: slate-ek relevancia szerint rendezve (label leakage!)")
    print(f"   → FALLBACK STRATÉGIA: baseline_query_metrics.csv használata")
    
    # Fallback: használjuk a baseline metrics CSV adatait
    if baseline_metrics_dict:
        print(f"\n   ✅ Baseline metrics elérhető, kombinált nehézségi score használata")
        slate_baseline_ndcgs_fallback = []
        
        for s in slate_baseline_ndcgs:
            query_id = s["query_id"]
            metrics = baseline_metrics_dict.get(query_id, {})
            
            # Kombinált nehézségi score: több metrika átlaga
            combined_score = (
                metrics.get("ndcg@10", 0.0) * 0.5 +
                metrics.get("mrr@5", 0.0) * 0.3 +
                metrics.get("recall@20", 0.0) * 0.2
            )
            
            slate_baseline_ndcgs_fallback.append({
                "query_id": query_id,
                "baseline_ndcg": combined_score,  # Kombinált score
                "num_relevant": s["num_relevant"],
                "num_high_relevant": s["num_high_relevant"],
            })
        
        slate_baseline_ndcgs = slate_baseline_ndcgs_fallback
        ndcgs = [s["baseline_ndcg"] for s in slate_baseline_ndcgs]
        
        print(f"   📊 Fallback score statisztikák:")
        print(f"      Átlag: {np.mean(ndcgs):.4f}")
        print(f"      Szórás: {np.std(ndcgs):.4f}")
        print(f"      Range: {np.max(ndcgs) - np.min(ndcgs):.4f}")
    else:
        print(f"\n   ⚠️  Baseline metrics nem elérhető!")
        print(f"   → EGYSZERŰSÍTETT FALLBACK: relevance count alapú nehézség")
        
        for s in slate_baseline_ndcgs:
            # Heurisztika: mennyi releváns dokumentum van?
            num_rel = s["num_relevant"]
            num_high = s["num_high_relevant"]
            # Normalizált score: több releváns = könnyebb
            s["baseline_ndcg"] = (num_high * 2 + num_rel) / 20.0  # max ~20
        
        ndcgs = [s["baseline_ndcg"] for s in slate_baseline_ndcgs]
        print(f"   📊 Heurisztikus score statisztikák:")
        print(f"      Átlag: {np.mean(ndcgs):.4f}")
        print(f"      Range: {np.max(ndcgs) - np.min(ndcgs):.4f}")

# Percentilis számítás
ndcg_percentiles = {
    "easy": np.percentile(ndcgs, 75),      # Top 25% - legjobb baseline (nDCG > ~0.5-0.6)
    "medium": np.percentile(ndcgs, 50),    # Top 50% - közepes baseline (nDCG > ~0.3-0.4)
    "hard": np.percentile(ndcgs, 25),      # Top 75% - nehéz baseline (nDCG > ~0.1-0.2)
    "very_hard": 0.0,                      # Minden - a legnehezebbek is
}

print(f"\n📈 Curriculum szintek (percentile):")
print(f"  Easy (> {ndcg_percentiles['easy']:.3f}): Top 25% - legjobb baseline slate-ek")
print(f"  Medium (> {ndcg_percentiles['medium']:.3f}): Top 50% - közepes baseline")
print(f"  Hard (> {ndcg_percentiles['hard']:.3f}): Top 75% - nehezebb slate-ek")
print(f"  Very Hard: Minden slate")

# 3. Slate-ek kategorizálása
slate_difficulty = {}
for s in slate_baseline_ndcgs:
    ndcg = s["baseline_ndcg"]
    if ndcg >= ndcg_percentiles["easy"]:
        difficulty = "easy"
    elif ndcg >= ndcg_percentiles["medium"]:
        difficulty = "medium"
    elif ndcg >= ndcg_percentiles["hard"]:
        difficulty = "hard"
    else:
        difficulty = "very_hard"
    slate_difficulty[s["query_id"]] = difficulty

# 4. Dataset építése difficulty tagging-gel
training_examples = []
slate_lookup = {}

for slate_data in slates_data:
    query_id = slate_data["query_id"]
    if query_id not in slate_difficulty:
        continue
    
    prompt = create_training_prompt(query_id, slate_data["slate"])
    difficulty = slate_difficulty[query_id]
    baseline_ndcg = next(s["baseline_ndcg"] for s in slate_baseline_ndcgs if s["query_id"] == query_id)
    
    training_examples.append({
        "prompt": prompt,
        "difficulty": difficulty,
        "baseline_ndcg": baseline_ndcg,
    })
    slate_lookup[query_id] = slate_data["slate"]

# 5. Difficulty eloszlás
difficulty_counts = {}
for ex in training_examples:
    d = ex["difficulty"]
    difficulty_counts[d] = difficulty_counts.get(d, 0) + 1

print(f"\n📊 Difficulty eloszlás:")
for diff in ["easy", "medium", "hard", "very_hard"]:
    count = difficulty_counts.get(diff, 0)
    pct = 100 * count / len(training_examples) if training_examples else 0
    avg_ndcg = np.mean([ex["baseline_ndcg"] for ex in training_examples if ex["difficulty"] == diff]) if count > 0 else 0
    print(f"  {diff:12s}: {count:3d} query ({pct:5.1f}%) - avg baseline nDCG: {avg_ndcg:.4f}")

# 6. Curriculum learning konfiguráció (használva a cella 4-ben definiált változókat)
# A CURRICULUM_ENABLED és CURRICULUM_STRATEGY már definiálva van a cella 4-ben

print(f"\n✅ Curriculum Learning konfiguráció:")
print(f"  Enabled: {CURRICULUM_ENABLED}")
print(f"  Strategy: {CURRICULUM_STRATEGY}")
if CURRICULUM_STRATEGY == "filtered_dataset":
    easy_count = len([ex for ex in training_examples if ex['difficulty'] == 'easy'])
    medium_count = len([ex for ex in training_examples if ex['difficulty'] == 'medium'])
    print(f"  Training dataset: easy ({easy_count}) + medium ({medium_count}) = {easy_count + medium_count} query")
    print(f"  Ez biztosítja, hogy csak jó baseline nDCG-val rendelkező példákkal tanítunk")
    print(f"  → Jobb tanulási signal → Jobb végső teljesítmény → Közelebb az 1.0 nDCG-hoz")

# 7. Train/eval split (stratified by difficulty ha lehet)
from collections import defaultdict

# Eval set: random 20%, de próbáljuk difficulty-t is megőrizni
full_dataset = Dataset.from_list(training_examples)
indices = np.arange(len(full_dataset))
train_indices, eval_indices = train_test_split(indices, test_size=1.0 - TRAIN_SPLIT, random_state=SEED, shuffle=True)

train_dataset = full_dataset.select(train_indices)
eval_dataset = full_dataset.select(eval_indices)

print(f"\n✅ Dataset létrehozva:")
print(f"  Training: {len(train_dataset)} query (80%)")
print(f"  Evaluation: {len(eval_dataset)} query (20%)")
print(f"  Slate lookup: {len(slate_lookup)} entry")

# 8. Training examples difficulty statisztikák
train_difficulty_dist = defaultdict(int)
for idx in train_indices:
    train_difficulty_dist[training_examples[idx]["difficulty"]] += 1

print(f"\n📊 Training set difficulty eloszlás:")
for diff in ["easy", "medium", "hard", "very_hard"]:
    count = train_difficulty_dist.get(diff, 0)
    print(f"  {diff:12s}: {count:3d} query")

print("\n" + "="*80)
print("💡 CURRICULUM LEARNING használata:")
print("   A training fokozatosan épül fel: először easy query-kkel,")
print("   aztán fokozatosan hozzáadjuk a nehezebbeket.")
print("   Ez segít a modellnek jobban tanulni a rangsorolást.")
print("="*80)

In [ ]:
# Eval split exportálása összehasonlító értékeléshez
# CRITICAL: Az eval split query_id-k exportálása determinisztikusan reprodukálható formátumban
# Ez biztosítja, hogy a baseline és GRPO modell ugyanazon eval halmazon legyen kiértékelve

print("\n" + "="*80)
print("📤 Eval Split Exportálása")
print("="*80)

# Query ID-k kinyerése az eval_dataset-ből
eval_query_ids = []

for example in eval_dataset:
    prompt = example["prompt"]
    # Ugyanaz a regex parsing logika, mint a reward function-ben
    match = re.search(r'(?:SEARCH QUERY|Query)[:\s"]*([^"\n]+)', prompt)
    query_id = None
    if match:
        query_id = match.group(1)
    else:
        match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
        if match2:
            query_id = match2.group(1)
    
    if query_id and query_id not in eval_query_ids:
        eval_query_ids.append(query_id)

# Rendezés a determinisztikus kimenetért
eval_query_ids.sort()

print(f"\n📊 Eval split statisztikák:")
print(f"  Eval query-k száma: {len(eval_query_ids)}")
print(f"  Train query-k száma: {len(train_dataset)}")
print(f"  Split arány: {len(eval_query_ids) / (len(eval_query_ids) + len(train_dataset)):.1%}")

# Eval split export JSON formátumban
eval_split_data = {
    "query_ids": eval_query_ids,
    "num_queries": len(eval_query_ids),
    "split_params": {
        "train_split": TRAIN_SPLIT,
        "seed": SEED,
        "shuffle": True
    },
    "created_at": pd.Timestamp.now().isoformat()
}

eval_split_path = OUTPUT_DIR / "eval_split.json"
with open(eval_split_path, 'w', encoding='utf-8') as f:
    json.dump(eval_split_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Eval split exportálva: {eval_split_path}")
print(f"   Ez a fájl használva lesz a baseline és GRPO összehasonlító notebookban")
print(f"   ugyanazon eval halmazon való kiértékeléshez.")

# Opcionális: CSV export is (egyszerűbb betöltéshez)
eval_split_csv = OUTPUT_DIR / "eval_split.csv"
eval_df = pd.DataFrame({"query_id": eval_query_ids})
eval_df.to_csv(eval_split_csv, index=False, encoding='utf-8')
print(f"✅ Eval split CSV: {eval_split_csv}")

print("\n" + "="*80)


In [ ]:
# Model betöltése - Memory Efficient
print(f"🔄 Model betöltése Unsloth-tal: {MODEL_NAME}")
print(f"  ⚙️  GPU Memory Utilization: {GPU_MEMORY_UTILIZATION}")
print(f"  ⚙️  Max LoRA Rank: {LORA_RANK}")

# Memória tisztítás model betöltés előtt
gc.collect()
torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=None,
    fast_inference=True,  # CRITICAL: Enable vLLM for GRPO rollouts (2-3x faster inference)
    max_lora_rank=LORA_RANK,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
    max_model_len=MAX_SEQ_LENGTH,  # CRITICAL: vLLM max_model_len must match max_seq_length for long prompts
    token=HF_TOKEN,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
    use_rslora=True,
    random_state=SEED,
)

# Tokenizer padding fix (tensor shape stability)
# CRITICAL: Csak padding_side állítása, NE állítsuk át a pad_token-et, mert a tokenizer újraállítása
# tenzorméret eltérést okozhat a GRPO trainer vLLM belső logikájával
tokenizer.padding_side = "right"
# Note: pad_token beállítása kihagyva - a GRPO trainer és vLLM automatikusan kezeli
# Az explicit pad_token átállítás ütközhet a modell config-tal és tenzor méret problémákat okozhat

print("✅ Model és tokenizer betöltve (Unsloth + vLLM + RSLoRA)")
print(f"  ✓ fast_inference=True: vLLM engedélyezve GRPO rollouts-hoz (2-3x gyorsabb)")
print(f"  ✓ Padding side: {tokenizer.padding_side}")
print(f"  ✓ Pad token ID: {tokenizer.pad_token_id if hasattr(tokenizer, 'pad_token_id') and tokenizer.pad_token_id else 'N/A (auto-managed)'}")
print(f"  ✓ Note: GRPO trainer automatikusan kezeli a tokenizer beállításokat vLLM-mel")

In [ ]:
# GPU memória monitoring model betöltés után
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    free = total - allocated
    
    print(f"\n📊 GPU Memória Model Betöltés Után:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")
    print(f"  Free: {free:.2f} GB")
    print(f"  Free %: {(free/total)*100:.1f}%")
    
    if free < 10:
        print(f"  ⚠️  FIGYELEM: Kevés szabad memória ({free:.2f} GB)!")
        print(f"  💡 Opció: További csökkentés lehet szükséges")


In [ ]:
# ====== MULTI-METRIKA REWARD FUNCTION (Sigmoid-Based) ======
# Konvex kombinációja három fő metrikának sigmoid transzformációval:
# - nDCG@10 (60%): Rangsor minőség (növelt súly)
# - MRR@5 (30%): Első releváns találat (gyakorlati használhatóság)
# - Recall@20 (10%): Completeness
#
# Javítások:
# - Sigmoid(delta) → stabilabb gradiens, nem saturálódik
# - MRR tie-break bonus: ha nDCG/Recall stagnál, de MRR javul → +0.02
# - Diverzitás büntetés: homogén top-10 (court/domain) → -ε

# Reward konfiguráció (config.py alapján)
REWARD_WEIGHTS = {
    "ndcg10": 0.60,    # 50% → 60%
    "mrr5": 0.30,      # 35% → 30%
    "recall20": 0.10,  # 15% → 10%
}
REWARD_CLIP_RANGE = (-1.0, 1.0)
REWARD_USE_SIGMOID = True
REWARD_MRR_TIEBREAK_BONUS = 0.02
REWARD_DIVERSITY_PENALTY = 0.05  # Diverzitás büntetés súlya

# Komponens tracking globális változó (logging célokra)
reward_components_log = []

def multi_metric_reward_function(completions, prompts, **kwargs):
    """
    Multi-metrika GRPO reward függvény sigmoid transzformációval.
    
    A reward három metrika súlyozott kombinációja:
    1. nDCG@10 (60%): Rangsor minőség
    2. MRR@5 (30%): Első releváns találat pozíciója (UX prioritás!)
    3. Recall@20 (10%): Releváns találatok lefedettsége
    
    Speciális funkciók:
    - Sigmoid(delta): stabil gradiens, nem saturálódik
    - MRR tie-break: +bonus ha csak MRR javul (felhasználói élmény++)
    - Diverzitás check: homogén top-10 → penalty
    
    Returns:
        List[float]: Reward értékek minden completion-höz
    """
    global reward_components_log
    rewards = []
    batch_components = []
    
    for completion, prompt in zip(completions, prompts):
        try:
            # 1. Query ID extraction
            match = re.search(r'(?:SEARCH QUERY|Query)[:\s"]*([^"\n]+)', prompt)
            query_id = None
            if match:
                query_id = match.group(1)
            else:
                match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
                if match2:
                    query_id = match2.group(1)
            
            if not query_id or query_id not in slate_lookup:
                # Parsing hiba - enyhe büntetés
                rewards.append(-0.1)
                batch_components.append({"error": "query_id_not_found"})
                continue
            
            # 2. Slate lookup
            slate = slate_lookup[query_id]
            
            # 3. Parse model output
            predicted_indices = parse_model_ranking(completion, len(slate))
            
            # 4. Baseline metrikák lookup
            if query_id not in baseline_metrics_dict:
                # Nincs baseline adat - fallback to zero baseline (nem ideális, de biztonságos)
                baseline_metrics = {
                    "ndcg@10": 0.0,
                    "mrr@5": 0.0,
                    "mrr@10": 0.0,  # Kompatibilitás
                    "recall@20": 0.0,
                }
            else:
                baseline_metrics = baseline_metrics_dict[query_id]

        
            
            # 5. Multi-metric reward számítás (helper function from src.reward_metrics)
            reward, components = compute_multi_metric_reward(
                predicted_indices=predicted_indices,
                slate=slate,
                baseline_metrics=baseline_metrics,
                weights=REWARD_WEIGHTS,
                clip_range=REWARD_CLIP_RANGE,
                use_sigmoid=REWARD_USE_SIGMOID,
                mrr_tiebreak_bonus=REWARD_MRR_TIEBREAK_BONUS,
                diversity_penalty_weight=REWARD_DIVERSITY_PENALTY,
            )
            
            rewards.append(reward)
            batch_components.append(components)
            
        except Exception as e:
            # Exception esetén enyhe büntetés
            rewards.append(-0.1)
            batch_components.append({"error": str(e)[:100]})
    
    # Komponensek mentése logginghoz
    reward_components_log.extend(batch_components)
    
    return rewards

print("✅ Multi-Metrika GRPO Reward function definiálva (Sigmoid-Based v2.0)")
print(f"   📊 Reward súlyok:")
print(f"      • nDCG@10:   {REWARD_WEIGHTS['ndcg10']:.0%} (rangsor minőség)")
print(f"      • MRR@5:     {REWARD_WEIGHTS['mrr5']:.0%} (első releváns találat)")
print(f"      • Recall@20: {REWARD_WEIGHTS['recall20']:.0%} (completeness)")
print(f"   🎯 Sigmoid transzformáció: {REWARD_USE_SIGMOID}")
print(f"   🎁 MRR tie-break bonus: +{REWARD_MRR_TIEBREAK_BONUS}")
print(f"   🌈 Diverzitás büntetés: {REWARD_DIVERSITY_PENALTY}")
print(f"   📏 Reward clipping: {REWARD_CLIP_RANGE}")
print(f"   📈 Komponens tracking: Enabled")

In [ ]:
# ====== DRY RUN VALIDATION - Korai ellenőrzés teljes tréning előtt ======
# Cél: 2-3 perc alatt validáljuk a rendszert (prompt, parsing, reward function, model output)
# Ez megakadályozza, hogy 25-35 percig futson a tréning hibás konfigurációval

print("="*80)
print("🔬 DRY RUN VALIDATION - Korai ellenőrzés")
print("="*80)
print("Ez a lépés 2-3 perc alatt teszteli a rendszert teljes tréning előtt.\n")

# Konfiguráció
DRY_RUN_SAMPLE_SIZE = 10  # 10 query tesztelése
DRY_RUN_MAX_NEW_TOKENS = 50  # Rövid generálások (gyors teszt)
DRY_RUN_PARSING_SUCCESS_THRESHOLD = 0.8  # 80% parsing success szükséges
DRY_RUN_MIN_REWARD_MEAN = -0.1  # Minimum reward mean (nagyon laza, csak validálás)

# 1. Mini-batch tesztelés: sample query-k kiválasztása
print("📊 1. Mini-batch tesztelés előkészítése...")
test_indices = np.random.choice(len(slates_data), size=min(DRY_RUN_SAMPLE_SIZE, len(slates_data)), replace=False)
test_slates = [slates_data[i] for i in test_indices]
test_prompts = [create_training_prompt(slate["query_id"], slate["slate"]) for slate in test_slates]

print(f"   ✅ {len(test_slates)} query kiválasztva teszteléshez")
print(f"   📋 Query ID példák:")
for i, slate in enumerate(test_slates[:3]):
    print(f"      {i+1}. {slate['query_id'][:60]}...")

# 2. Prompt parsing validáció
print(f"\n📝 2. Prompt parsing validáció...")
parsing_successes = 0
parsing_failures = []

for i, (prompt, slate_data) in enumerate(zip(test_prompts, test_slates)):
    # Teszteljük a query_id extraction-t (ugyanaz mint a reward function-ben)
    match = re.search(r'(?:SEARCH QUERY|Query)[:\s"]*([^"\n]+)', prompt)
    query_id = None
    if match:
        query_id = match.group(1)
    else:
        match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
        if match2:
            query_id = match2.group(1)
    
    if query_id and query_id in slate_lookup:
        parsing_successes += 1
    else:
        parsing_failures.append({
            "index": i,
            "query_id_expected": slate_data["query_id"],
            "query_id_extracted": query_id,
        })

parsing_success_rate = parsing_successes / len(test_prompts) if test_prompts else 0.0
print(f"   ✅ Parsing success rate: {parsing_success_rate:.2%} ({parsing_successes}/{len(test_prompts)})")

if parsing_failures:
    print(f"   ⚠️  {len(parsing_failures)} parsing hiba:")
    for failure in parsing_failures[:3]:
        print(f"      Query {failure['index']}: expected '{failure['query_id_expected'][:50]}...', got '{failure['query_id_extracted']}'")

# 3. Slate struktúra validáció
print(f"\n📦 3. Slate struktúra validáció...")
slate_issues = []
relevance_distributions = []

for i, slate_data in enumerate(test_slates):
    slate = slate_data.get("slate", [])
    if not slate:
        slate_issues.append({"index": i, "issue": "empty_slate"})
        continue
    
    relevances = [doc.get('relevance', 0) for doc in slate]
    relevant_count = sum(1 for r in relevances if r >= 1)
    high_rel_count = sum(1 for r in relevances if r == 2)
    
    relevance_distributions.append({
        "total": len(relevances),
        "relevant": relevant_count,
        "high_relevant": high_rel_count,
        "baseline_ndcg": calculate_ndcg(list(range(len(relevances))), relevances, k=10),
    })
    
    if relevant_count == 0:
        slate_issues.append({"index": i, "issue": "no_relevant_docs", "query": slate_data["query_id"][:50]})

if slate_issues:
    print(f"   ⚠️  {len(slate_issues)} slate probléma található:")
    for issue in slate_issues[:3]:
        print(f"      Query {issue['index']}: {issue['issue']}")
else:
    print(f"   ✅ Minden slate valid")

avg_stats = {
    "total_docs": np.mean([d["total"] for d in relevance_distributions]),
    "relevant_docs": np.mean([d["relevant"] for d in relevance_distributions]),
    "high_relevant_docs": np.mean([d["high_relevant"] for d in relevance_distributions]),
    "baseline_ndcg": np.mean([d["baseline_ndcg"] for d in relevance_distributions]),
}
print(f"   📊 Átlagos slate statisztikák:")
print(f"      Dokumentumok/slate: {avg_stats['total_docs']:.1f}")
print(f"      Releváns dokumentumok: {avg_stats['relevant_docs']:.1f}")
print(f"      Magas relevancia (rel=2): {avg_stats['high_relevant_docs']:.1f}")
print(f"      Baseline nDCG@10: {avg_stats['baseline_ndcg']:.4f}")

# 4. Model inference teszt (ha modell már betöltve)
print(f"\n🤖 4. Model inference teszt...")
try:
    model.eval()
    test_completions = []
    
    # Egyetlen prompt tesztelése (gyors)
    test_prompt = test_prompts[0]
    inputs = tokenizer(test_prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=DRY_RUN_MAX_NEW_TOKENS,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    completion = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    test_slate = test_slates[0]
    parsed_indices = parse_model_ranking(completion, len(test_slate["slate"]))
    
    is_valid = len(parsed_indices) == len(test_slate["slate"]) and len(set(parsed_indices)) == len(parsed_indices)
    
    print(f"   ✅ Model inference sikeres")
    print(f"      Completion hossz: {len(completion)} karakter")
    print(f"      Parsed indices: {len(parsed_indices)}/{len(test_slate['slate'])}")
    print(f"      Valid parsing: {is_valid}")
    print(f"      Sample output: {completion[:100]}...")
    
    model.train()
    
except Exception as e:
    print(f"   ⚠️  Model inference hiba (ez oké, ha modell még nincs betöltve): {e}")

# 5. Reward function tesztelés
print(f"\n🎯 5. Reward function tesztelés...")
test_rewards = []
test_reward_details = []

for prompt, slate_data in zip(test_prompts[:5], test_slates[:5]):  # Csak 5 query teljes teszt
    try:
        # Szimulált completion (random ranking a teszthez)
        test_completion = ",".join(str(i) for i in np.random.permutation(len(slate_data["slate"])))
        
        # Reward számítás (ugyanaz a logika mint a reward_function-ben)
        query_id = slate_data["query_id"]
        slate = slate_lookup[query_id]
        relevance = [doc.get('relevance', 0) for doc in slate]
        baseline = list(range(len(slate)))
        predicted = parse_model_ranking(test_completion, len(slate))
        
        ndcg_baseline = calculate_ndcg(baseline, relevance, k=NDCG_K)
        ndcg_policy = calculate_ndcg(predicted, relevance, k=NDCG_K)
        reward = ndcg_policy - ndcg_baseline
        
        test_rewards.append(reward)
        test_reward_details.append({
            "ndcg_baseline": ndcg_baseline,
            "ndcg_policy": ndcg_policy,
            "reward": reward,
        })
    except Exception as e:
        test_rewards.append(-0.5)
        test_reward_details.append({"error": str(e)})

if test_rewards:
    reward_mean = np.mean(test_rewards)
    reward_std = np.std(test_rewards)
    print(f"   ✅ Reward function működik")
    print(f"      Reward mean: {reward_mean:.4f}")
    print(f"      Reward std: {reward_std:.4f}")
    print(f"      Reward range: [{min(test_rewards):.4f}, {max(test_rewards):.4f}]")
else:
    print(f"   ⚠️  Reward function tesztelés nem sikerült")

# 6. Összefoglaló és döntés
print(f"\n" + "="*80)
print("📋 DRY RUN VALIDATION ÖSSZEFOGLALÓ")
print("="*80)

validation_passed = True
warnings = []

if parsing_success_rate < DRY_RUN_PARSING_SUCCESS_THRESHOLD:
    validation_passed = False
    warnings.append(f"❌ Parsing success rate túl alacsony: {parsing_success_rate:.2%} < {DRY_RUN_PARSING_SUCCESS_THRESHOLD:.0%}")

if len(slate_issues) > len(test_slates) * 0.3:  # Több mint 30% slate probléma
    validation_passed = False
    warnings.append(f"❌ Túl sok slate probléma: {len(slate_issues)}/{len(test_slates)}")

if test_rewards and np.mean(test_rewards) < DRY_RUN_MIN_REWARD_MEAN:
    warnings.append(f"⚠️  Alacsony reward mean: {np.mean(test_rewards):.4f} (nem blokkolja)")

if avg_stats['baseline_ndcg'] < 0.1:
    warnings.append(f"⚠️  Alacsony baseline nDCG: {avg_stats['baseline_ndcg']:.4f} (lehet, hogy rossz slate minőség)")

print("\n✅ Pozitív mutatók:")
print(f"   ✓ Parsing success: {parsing_success_rate:.2%}")
print(f"   ✓ Átlagos baseline nDCG@10: {avg_stats['baseline_ndcg']:.4f}")
print(f"   ✓ Átlagos releváns dokumentumok/slate: {avg_stats['relevant_docs']:.1f}")

if warnings:
    print("\n⚠️  Figyelmeztetések:")
    for warning in warnings:
        print(f"   {warning}")

if validation_passed:
    print("\n✅ DRY RUN VALIDATION PASSED - Training folytatható!")
    print("   A rendszer validnak tűnik, folytathatod a teljes tréninget.")
else:
    print("\n❌ DRY RUN VALIDATION FAILED - Training NEM AJÁNLOTT!")
    print("   Kérlek javítsd a hibákat a tréning indítása előtt.")
    print("   Ellenőrizd:")
    print("   - Prompt generálás és query_id extraction")
    print("   - Slate struktúra és relevancia eloszlás")
    print("   - Reward function működés")

print("\n" + "="*80)


In [ ]:
# GRPO Trainer konfiguráció - MEMORY EFFICIENT + CURRICULUM LEARNING
grpo_config = GRPOConfig(
    output_dir=str(OUTPUT_DIR),
    max_steps=MAX_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_generations=NUM_GENERATIONS,
    generation_batch_size=GENERATION_BATCH_SIZE,  # Explicit: LCM(6, 2) = 6
    optim=OPTIMIZER_NAME,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    loss_type="dr_grpo",
    scale_rewards="batch",
    use_vllm=True,
    vllm_importance_sampling_correction=False,  # vLLM token logprobs nem egyeznek az alap modellével
    importance_sampling_level="token",
    mask_truncated_completions=True,
    epsilon=0.1,  # Enhanced: 0.2 → 0.1 (smaller policy updates, more stable)
    bf16=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False, "use_unsloth": True},
    max_grad_norm=1.0,
    logging_steps=LOGGING_STEPS,
    logging_first_step=True,
    eval_steps=EVAL_STEPS,
    save_steps=SAVE_STEPS,
    dataloader_num_workers=1,  # 2→1: csökkentett worker
    seed=SEED,
    max_completion_length=768,  # CRITICAL: explicit completion length (default 256 túl kevés 30 dokumentumhoz)
)

grpo_config.unsloth_num_chunks = 1

print("\n✅ GRPO Trainer konfiguráció kész (Memory Efficient + Curriculum Learning)")
print(f"  ✓ generation_batch_size={GENERATION_BATCH_SIZE}")
print(f"    - osztható num_generations={NUM_GENERATIONS}-szel: {GENERATION_BATCH_SIZE}÷{NUM_GENERATIONS}={GENERATION_BATCH_SIZE//NUM_GENERATIONS}")
print(f"    - osztható per_device_batch={PER_DEVICE_BATCH_SIZE}-gyel: {GENERATION_BATCH_SIZE}÷{PER_DEVICE_BATCH_SIZE}={GENERATION_BATCH_SIZE//PER_DEVICE_BATCH_SIZE}")
print(f"  ✓ Effective batch size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  ✓ Memória optimalizáció: 50% GPU util, 1 worker, rank 32")
if CURRICULUM_ENABLED:
    print(f"  ✓ Curriculum Learning: {CURRICULUM_STRATEGY} stratégia aktiválva")

## 🧪 Multi-Metrika Reward Validation

Most teszteljük a frissített multi-metrika reward függvényt:
- Sigmoid transzformáció működése
- Komponensek (nDCG/MRR/Recall) számítása
- MRR tie-break bonus
- Diverzitás check
- Baseline metrics betöltés

In [ ]:
# ====== MULTI-METRIKA REWARD VALIDATION ======
# Gyors teszt: multi-metric reward function működik-e helyesen

print("="*80)
print("🧪 MULTI-METRIKA REWARD VALIDÁCIÓ")
print("="*80)

# Teszt konfiguráció
NUM_TEST_QUERIES = 5
test_results = []

print(f"\n📊 {NUM_TEST_QUERIES} query tesztelése multi-metrika reward-dal...\n")

for i, slate_data in enumerate(slates_data[:NUM_TEST_QUERIES]):
    query_id = slate_data["query_id"]
    slate = slate_data["slate"]
    
    print(f"🔍 Test {i+1}/{NUM_TEST_QUERIES}: {query_id[:60]}...")
    
    # 1. Baseline metrikák check
    if query_id in baseline_metrics_dict:
        baseline = baseline_metrics_dict[query_id]
        print(f"   ✅ Baseline metrikák: nDCG@10={baseline.get('ndcg@10', 0):.4f}, MRR@5={baseline.get('mrr@5', 0):.4f}")
    else:
        print(f"   ⚠️  Nincs baseline metrika erre a query-re (fallback to zero)")
        baseline = {"ndcg@10": 0.0, "mrr@5": 0.0, "recall@20": 0.0}
    
    # 2. Szimulált policy ranking (random permutation)
    predicted_indices = list(np.random.permutation(len(slate)))
    
    # 3. Multi-metric reward számítás
    try:
        reward, components = compute_multi_metric_reward(
            predicted_indices=predicted_indices,
            slate=slate,
            baseline_metrics=baseline,
            weights=REWARD_WEIGHTS,
            clip_range=REWARD_CLIP_RANGE,
            use_sigmoid=REWARD_USE_SIGMOID,
            mrr_tiebreak_bonus=REWARD_MRR_TIEBREAK_BONUS,
            diversity_penalty_weight=REWARD_DIVERSITY_PENALTY,
        )
        
        # 4. Eredmények
        print(f"   📈 Policy metrikák:")
        print(f"      nDCG@10:   {components['policy_ndcg10']:.4f} (baseline: {components['baseline_ndcg10']:.4f}, Δ={components['delta_ndcg10']:+.4f})")
        print(f"      MRR@5:     {components['policy_mrr5']:.4f} (baseline: {components['baseline_mrr5']:.4f}, Δ={components['delta_mrr5']:+.4f})")
        print(f"      Recall@20: {components['policy_recall20']:.4f} (baseline: {components['baseline_recall20']:.4f}, Δ={components['delta_recall20']:+.4f})")
        
        print(f"   🎯 Reward komponensek:")
        print(f"      nDCG comp:  {components['component_ndcg10']:+.4f} (60% súly)")
        print(f"      MRR comp:   {components['component_mrr5']:+.4f} (30% súly)")
        print(f"      Recall comp:{components['component_recall20']:+.4f} (10% súly)")
        
        print(f"   💰 Total reward: {components['reward_total']:.4f} (raw: {components['reward_raw']:.4f})")
        
        test_results.append({
            "query_id": query_id,
            "reward": reward,
            "components": components,
            "success": True,
        })
        
    except Exception as e:
        print(f"   ❌ Hiba: {e}")
        test_results.append({
            "query_id": query_id,
            "error": str(e),
            "success": False,
        })
    
    print()

# Összefoglaló
success_count = sum(1 for r in test_results if r.get("success", False))
success_rate = success_count / len(test_results) if test_results else 0.0

print("="*80)
print("📊 VALIDÁCIÓ ÖSSZEFOGLALÓ")
print("="*80)
print(f"\n✅ Sikeres reward számítások: {success_count}/{len(test_results)} ({success_rate:.0%})")

if success_count > 0:
    successful_rewards = [r["reward"] for r in test_results if r.get("success")]
    print(f"\n📈 Reward statisztikák:")
    print(f"   Mean:   {np.mean(successful_rewards):.4f}")
    print(f"   Std:    {np.std(successful_rewards):.4f}")
    print(f"   Min:    {np.min(successful_rewards):.4f}")
    print(f"   Max:    {np.max(successful_rewards):.4f}")
    print(f"   Median: {np.median(successful_rewards):.4f}")
    
    # Komponensek átlaga
    if test_results[0].get("components"):
        avg_delta_ndcg = np.mean([r["components"]["delta_ndcg10"] for r in test_results if r.get("success")])
        avg_delta_mrr = np.mean([r["components"]["delta_mrr5"] for r in test_results if r.get("success")])
        avg_delta_recall = np.mean([r["components"]["delta_recall20"] for r in test_results if r.get("success")])
        
        print(f"\n📊 Átlagos delta metrikák:")
        print(f"   Δ nDCG@10:   {avg_delta_ndcg:+.4f}")
        print(f"   Δ MRR@5:     {avg_delta_mrr:+.4f}")
        print(f"   Δ Recall@20: {avg_delta_recall:+.4f}")

if success_rate >= 0.8:
    print(f"\n✅ MULTI-METRIKA REWARD VALIDATION PASSED!")
    print(f"   A reward function működik, folytatható a training.")
else:
    print(f"\n❌ MULTI-METRIKA REWARD VALIDATION FAILED!")
    print(f"   Ellenőrizd a baseline metrics betöltését és a reward function implementációt.")

print("="*80)

In [ ]:
# ====== CURRICULUM LEARNING - Simplified Approach ======
# Mivel a TRL trainer nem támogatja a dataset dinamikus változtatását közben,
# két megoldás van:
# 1. Weighted sampling: könnyebb példákat gyakrabban választjuk (implementálva)
# 2. Filtered dataset: csak az easy/medium példákat használjuk (simpler, de hatásos)

CURRICULUM_STRATEGY = "weighted_sampling"  # vagy "filtered_dataset"

if CURRICULUM_ENABLED and CURRICULUM_STRATEGY == "filtered_dataset":
    # Egyszerű stratégia: csak easy + medium példákat használunk (legjobb baseline nDCG)
    # Ez biztosítja, hogy a modell először megtanulja a jól rangsorolni, amikor van mit rangsorolni
    print("\n📊 Curriculum Learning: Filtered Dataset stratégia")
    print("   Csak easy + medium difficulty példákat használunk a training-ben")
    
    filtered_train_examples = [ex for ex in training_examples 
                              if ex["difficulty"] in ["easy", "medium"]]
    filtered_train_indices = [i for i in train_indices 
                             if training_examples[i]["difficulty"] in ["easy", "medium"]]
    
    train_dataset = full_dataset.select(filtered_train_indices)
    
    print(f"   Eredeti training examples: {len(training_examples)}")
    print(f"   Filtered training examples: {len(filtered_train_examples)} ({100*len(filtered_train_examples)/len(training_examples):.1f}%)")
    print(f"   Átlagos baseline nDCG (filtered): {np.mean([ex['baseline_ndcg'] for ex in filtered_train_examples]):.4f}")
    print(f"   vs. Eredeti átlag: {np.mean([ex['baseline_ndcg'] for ex in training_examples]):.4f}")

# Trainer inicializálása
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[multi_metric_reward_function],  # UPDATED: Multi-metric reward
    args=grpo_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

print("\n✅ GRPO Trainer inicializálva")
print(f"  ✓ Reward function: Multi-Metric (nDCG@10 + MRR@5 + Recall@20)")
if CURRICULUM_ENABLED:
    print(f"  ✓ Curriculum Learning: {CURRICULUM_STRATEGY}")
    print(f"  ✓ Training queries: {len(train_dataset)}")
else:
    print(f"  Training queries: {len(train_dataset)}")
print(f"  Eval queries: {len(eval_dataset)}")
print(f"  GPU memória: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

In [ ]:
# Training indítása
print("\n🚀 GRPO TRAINING INDÍTÁSA\n")
print("="*60)

if CURRICULUM_ENABLED:
    print(f"📚 Curriculum Learning aktív: {CURRICULUM_STRATEGY}")
    if CURRICULUM_STRATEGY == "filtered_dataset":
        print("   Training könnyebb példákkal (easy + medium) - jobb tanulási signal")
    print()

try:
    trainer.train()
    print("\n" + "="*60)
    print("✅ Training sikeresen befejezve!")
except Exception as e:
    print(f"\n❌ Training hiba: {e}")
    import traceback
    traceback.print_exc()
    raise

In [3]:
# Robusztus GRPO tréning vizualizáció (hibatűrő)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Megpróbáljuk a TRL trainer log_history-ját használni; ha nincs, esünk vissza a metrics.json-re
log_history = None
try:
    _ = trainer  # ellenőrzés, hogy létezik-e
    log_history = getattr(trainer.state, "log_history", None)
except NameError:
    log_history = None

# Ha nincs log_history, próbáljuk a metrics.json-ból kirajzolni legalább a jutalom trendet
if not log_history:
    metrics_path = METRICS_FILE if 'METRICS_FILE' in globals() else Path("data/models/grpo_policy/metrics.json")
    if Path(metrics_path).exists():
        with open(metrics_path, 'r', encoding='utf-8') as f:
            metrics = json.load(f)
        rewards_trend = metrics.get("training_rewards", {}).get("trend", [])
        if rewards_trend:
            plt.figure(figsize=(8, 4))
            plt.plot(rewards_trend, color="royalblue")
            plt.title("Jutalom trend (metrics.json)")
            plt.xlabel("Lépés")
            plt.ylabel("Jutalom")
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️ Nincs elérhető trainer.state.log_history és a metrics.json nem tartalmaz 'training_rewards.trend'-et.")
    else:
        print("⚠️ Nincs elérhető trainer.state.log_history és a metrics.json fájl sem található.")
else:
    # Innentől a log_history alapján dolgozunk
    logs = pd.DataFrame(log_history)
    if logs.empty:
        print("⚠️ A logok DataFrame üres.")
    else:
        # Csak numerikus oszlopok megtartása
        numeric_cols = [c for c in logs.columns if np.issubdtype(pd.Series(logs[c]).dropna().dtype, np.number)]
        if not numeric_cols:
            print("⚠️ A logokban nincs numerikus oszlop a megjelenítéshez.")
        else:
            logs = logs[numeric_cols].copy()

            def pick_col(df, candidates):
                for c in candidates:
                    if c in df.columns:
                        return c
                return None

            # Reward (átlag) oszlop kiválasztása több lehetséges kulccsal
            reward_candidates = [
                "rewards/reward_function/mean",  # TRL GRPO gyakori
                "rewards/mean",
                "reward/mean",
                "rewards/reward",
                "reward",
            ]
            reward_col = pick_col(logs, reward_candidates)

            # Reward szórás oszlop (opcionális)
            reward_std_candidates = [
                "rewards/reward_function/std",
                "rewards/std",
                "reward/std",
            ]
            reward_std_col = pick_col(logs, reward_std_candidates)

            # Loss és KL oszlopok (opcionális)
            loss_col = pick_col(logs, ["loss", "train/loss"])  
            kl_col = pick_col(logs, ["kl", "train/kl", "kl_divergence"]) 

            # Ábrák létrehozása dinamikusan az elérhető metrikák alapján
            fig, axes = plt.subplots(2, 2, figsize=(13, 8))
            axes = axes.flatten()
            ax_idx = 0

            # 1) Reward trend (simított) + opcionális szórás sáv
            if reward_col is not None:
                reward_smooth = logs[reward_col].rolling(window=10, min_periods=1).mean()
                axes[ax_idx].plot(reward_smooth, label=f"Jutalom (átlag, {reward_col})", color="royalblue")
                if reward_std_col is not None:
                    lo = reward_smooth - logs[reward_std_col]
                    hi = reward_smooth + logs[reward_std_col]
                    axes[ax_idx].fill_between(logs.index, lo, hi, alpha=0.2, color="royalblue", label="± szórás")
                axes[ax_idx].set_title("Jutalom trend")
                axes[ax_idx].set_xlabel("Lépés")
                axes[ax_idx].set_ylabel("Jutalom")
                axes[ax_idx].legend()
                ax_idx += 1
            else:
                print("ℹ️ Nem található reward átlag oszlop a logokban. Kihagyom a jutalom görbét.")

            # 2) Loss görbe
            if loss_col is not None and ax_idx < len(axes):
                axes[ax_idx].plot(logs[loss_col], label="Veszteség (loss)", color="firebrick")
                axes[ax_idx].set_title("Tanítási veszteség")
                axes[ax_idx].set_xlabel("Lépés")
                axes[ax_idx].set_ylabel("Loss")
                axes[ax_idx].legend()
                ax_idx += 1

            # 3) KL görbe
            if kl_col is not None and ax_idx < len(axes):
                axes[ax_idx].plot(logs[kl_col], label="KL divergencia", color="darkorange")
                axes[ax_idx].set_title("KL divergencia alakulása")
                axes[ax_idx].set_xlabel("Lépés")
                axes[ax_idx].set_ylabel("KL")
                axes[ax_idx].legend()
                ax_idx += 1

            # 4) Reward szórás külön (ha van és maradt hely)
            if reward_std_col is not None and ax_idx < len(axes):
                axes[ax_idx].plot(logs[reward_std_col], label="Jutalom szórás", color="seagreen")
                axes[ax_idx].set_title("Jutalom szórás")
                axes[ax_idx].set_xlabel("Lépés")
                axes[ax_idx].set_ylabel("Szórás")
                axes[ax_idx].legend()
                ax_idx += 1

            # Ha kevesebb metrika áll rendelkezésre, a maradék tengelyeket rejtsük el
            for i in range(ax_idx, len(axes)):
                axes[i].axis('off')

            plt.tight_layout()
            plt.show()


⚠️ Nincs elérhető trainer.state.log_history és a metrics.json fájl sem található.


In [ ]:
# OPTIONAL: Prompt Quality Check - Sample Completions
# Futtasd TRAINING ELŐTT, hogy lásd, milyen outputokat generál a modell

print("🔬 PROMPT QUALITY CHECK - Sample Completions Generálása\n")
print("Ez a cella teszteli, hogy a modell milyen outputokat generál az új prompttal.")
print("FONTOS: Ez optional, csak debug célra.\n")

# Egy random query kiválasztása
import random
random.seed(42)
test_slate_idx = random.randint(0, len(slates_data)-1)
test_slate_data = slates_data[test_slate_idx]
test_prompt_text = create_training_prompt(test_slate_data["query_id"], test_slate_data["slate"])

# Generálunk néhány completion-t
print(f"Query: {test_slate_data['query_id'][:80]}...")
print(f"Slate size: {len(test_slate_data['slate'])}")
print("-" * 80)

try:
    # Model inference mode
    model.eval()
    
    # Tokenize prompt
    inputs = tokenizer(test_prompt_text, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    print("\n🎲 3 Sample Completion Generálása...\n")
    
    for i in range(3):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=768,  # Növelve 100-ról 768-re, hogy elegendő legyen 30 dokumentum rangsorolásához
                do_sample=True,
                temperature=0.8,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,  # Stop sequence hozzáadása
            )
        
        completion = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        parsed = parse_model_ranking(completion, len(test_slate_data["slate"]))
        
        print(f"Completion {i+1}:")
        print(f"  Raw output: {completion[:150]}")
        print(f"  Parsed indices: {parsed[:10]}")  # First 10 indices
        print(f"  Valid? {len(parsed) == len(test_slate_data['slate']) and len(set(parsed)) == len(parsed)}")
        print()
    
    # Relevance info
    relevances = [doc.get('relevance', 0) for doc in test_slate_data["slate"]]
    print(f"📊 Ground truth relevance distribution:")
    print(f"  Relevant docs (rel>=1): {sum(1 for r in relevances if r >= 1)}/{len(relevances)}")
    print(f"  Highly relevant (rel=2): {sum(1 for r in relevances if r == 2)}/{len(relevances)}")
    print(f"  Baseline order nDCG@10: {calculate_ndcg(list(range(len(relevances))), relevances, k=10):.4f}")
    
    model.train()
    print("\n✅ Prompt quality check complete!")
    
except Exception as e:
    print(f"❌ Error during quality check: {e}")
    print("Ez nem probléma, a training ettől még futhat.")

print("\n" + "="*80)

In [ ]:
# Artifactumok mentése
print("\n💾 Artifactumok mentése...")
model.save_pretrained_merged(str(OUTPUT_DIR), tokenizer, save_method="lora")
print(f"  ✅ LoRA adapter: {OUTPUT_DIR}")

## 📊 Modell Összehasonlítás

Az értékelés és a baseline vs GRPO összehasonlítás a **`notebooks/model_comparison.ipynb`** notebookban történik.

Ez biztosítja, hogy a baseline és GRPO modell **ugyanazon eval halmazon** legyen kiértékelve, így a metrikák összevethetők.


In [ ]:
# NOTE: Az értékelés a notebooks/model_comparison.ipynb notebookban történik.
# Ez biztosítja, hogy a baseline és GRPO modell ugyanazon eval halmazon legyen kiértékelve.
